#### Setting up Wikipedia as tool

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)

wiki.name

'wikipedia'

#### Creating a Retrieval tool for custom web page

In [ ]:
import warnings
import sys
import os

# Ignore warnings
warnings.filterwarnings('ignore')

# Redirect stderr (where warnings often go)
sys.stderr = open(os.devnull, 'w')


from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents,OllamaEmbeddings())

retriever = vectordb.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027179654E20>, search_kwargs={})

In [6]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search",
                                       "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

retriever_tool.name

'langsmith_search'

#### Setting up ARXIV as a tool

In [7]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

arxiv.name

'arxiv'

In [8]:
tools = [wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Anaconda\\envs\\llm_venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002717AA47130>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vecto

#### Setting up the llm model - llama 2

In [9]:

'''
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")
llm
'''

from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama2")
llm



ChatOllama()

#### Importing prompt template from langchain hub

In [10]:
from langchain import hub

prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

#### Creating Agent

In [11]:
# Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [12]:
# Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) 

agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

#### Invoking Agent for responding to user input

In [13]:
#input = input("Please enter your enquiry - ")
agent_executor.invoke({"input":"What is Langchain"})



> Entering new AgentExecutor chain...
Langchain is an open-source platform designed to help organizations manage and analyze their language data. It provides a range of tools and features for collecting, cleaning, and analyzing text data, including natural language processing (NLP) tasks such as sentiment analysis, topic modeling, and named entity recognition.

Here are some key features of Langchain:

1. Data Collection: Langchain allows you to collect text data from various sources, including web pages, documents, social media platforms, and more. You can use pre-built integrations or create custom connectors to collect data from specific sources.
2. Data Cleaning: Langchain provides tools for cleaning and preprocessing text data, including tokenization, stopword removal, stemming, and lemmatization.
3. NLP Tasks: Langchain offers a range of NLP tasks, such as sentiment analysis, topic modeling, named entity recognition, and part-of-speech tagging. You can use these tasks to extrac

{'input': 'What is Langchain',
 'output': 'Langchain is an open-source platform designed to help organizations manage and analyze their language data. It provides a range of tools and features for collecting, cleaning, and analyzing text data, including natural language processing (NLP) tasks such as sentiment analysis, topic modeling, and named entity recognition.\n\nHere are some key features of Langchain:\n\n1. Data Collection: Langchain allows you to collect text data from various sources, including web pages, documents, social media platforms, and more. You can use pre-built integrations or create custom connectors to collect data from specific sources.\n2. Data Cleaning: Langchain provides tools for cleaning and preprocessing text data, including tokenization, stopword removal, stemming, and lemmatization.\n3. NLP Tasks: Langchain offers a range of NLP tasks, such as sentiment analysis, topic modeling, named entity recognition, and part-of-speech tagging. You can use these tasks 